In [1]:
import heapq as hq
from math import sqrt
import networkx as nx
import matplotlib.pyplot as plt
import random
from random import shuffle 

class Event:
    def __init__(self, time, action, BMName, passenger):
        self.time = time
        self.action = action
        self.bm = BMName
        self.p = passenger
    def __lt__(self, other):
        return (self.time, self.bm, self.p) < (other.time, other.bm, other.p)
    def __le__(self, other):
        return (self.time, self.bm, self.p) <= (other.time,  other.bm, other.p)

class Passagier:
    def __init__(self, Ziel):
        self.ziel = Ziel
        self.umstieg = 8 #'dummy_place'
        self.time = 0
        self.bm = 'dummy'
    def __lt__(self, other):
        return (self.time, self.bm, self.ziel, self.umstieg) < (other.time, other.bm, other.ziel, other.umstieg)
    def __le__(self, other):
        return (self.time, self.bm, self.ziel, self.umstieg) <= (other.time, other.bm, other.ziel, other.umstieg)

class Vehikel:
    def __init__(self, Name, Color, Capacity, Itinerary, TravelTimes):
        self.name = Name
        self.color = Color
        self.capacity = Capacity
        self.itinerary = Itinerary
        self.travel_times = TravelTimes
        self.n_passengers = []
        self.stop = 0

def shuffle0(passengers):
    for i in range(len(passengers)):
        x = [j for j in range(len(passengers[i]))]
        shuffle(x)
        passengers[i] = [passengers[i][k] for k in x]

def calc_pic(grah, ListeColor, ListeSize):
    nx.draw(graph, nx.get_node_attributes(graph,'pos'), with_labels = True, node_color = ListeColor, node_size = ListeSize)

def cap(menge):
    temp = []
    if len(menge) == 0 or len(menge)== 1:
        return menge
    elif len(menge) == 2:
        for i in menge[0]:
            if i in menge[1]:
                temp += [i]
    else:
        return print('Ein Fehler')
    return [temp]
#default='#ecf2e3'
default='#cad3b8' #nice color

In [2]:
# Lösung für Passagier als Klasse
def solve(eventq, passengers, bms, graph):
    TIME = []
    Schleife = []
    v = [len([idx for idx, passenger in enumerate(passengers[n]) if passenger.ziel == n]) == 0 for n in range(len(passengers))]
    if not (v == [True]*len(passengers)):
        ErrorMessage = 'Bei Passengers/Netz sind an den Knoten {} Passagiere, die auf derselben Ziel untergebracht sind.'.format([idx for idx, boolean in enumerate(v) if boolean == False])
        raise Exception(ErrorMessage)
    
    if not (len(graph.node)) == (len(passengers)):
        ErrorMessage = 'Die Graphiken werden fehlerhaft berechnet.\n'
        if len(graph.node) < len(passengers):
            ErrorMessage += 'Es sind mehr Einträge in Passengers ({}) als Knoten im Graph ({})'.format(len(passengers),len(graph.node)) + '\n'
            ErrorMessage += 'Löschen/Fügen Sie ({}) Einträge/Knoten im Passengers/Graph /ein'.format(abs(len(passengers) - len(graph.node)))
        else:
            ErrorMessage += 'Es sind weniger Einträge in Passengers ({}) als Knoten im Graph ({})'.format(len(passengers),len(graph.node)) + '\n'
            ErrorMessage += 'Fügen/Löschen Sie ({}) Einträge/Knoten im Passengers/Graph /ein'.format(abs(len(passengers) - len(graph.node)))
        raise Exception(ErrorMessage)
    
    #if not [(len(bms[i].itinerary == len(bms[i].travel_times)) for i in range(len(bms))] == [True]*len(bms):
    #    raise Exception('In der Liste BMS sind itinerary und travel_times nicht gleich groß.')
    
    zykel = 0
    while True:
        print('\n Durchläuft die While-Schleife zum {}. Mal'.format(zykel))
       
        ''' # Ausgabe der Queue mit Checks
        m='Länge der Queue: {} Queue: ['.format(len(eventq))
        for el in eventq:
            p = el.p
            m += '[Time: {}.{}, Aktion: {}, BMName:{}.{}]; '.format(el.time, el.time==p.time, el.action, el.bm, el.bm==p.bm)
        print(m+']')
        #'''
        
        # Abbruchbedingung
        print([len(bm.n_passengers) for bm in bms])
        print([len(node) for node in passengers])
        Vehiekel = [idx for idx, bahn in enumerate(bms) if bahn.name == eventq[0].bm]
        vehiekel = bms[Vehiekel[0]]
        Schleife += [{'Zykel': zykel,'PassagierVehiekl': [len(bm.n_passengers) for bm in bms],'PassagierKnoten': [len(node) for node in passengers], 'Time': eventq[0].time, 'Aktion': eventq[0].action, 'Vehiekel': vehiekel}]
        if (([len(node) == 0 for node in passengers] == [True]*len(passengers)) and 
           ([len(bm.n_passengers) == 0 for bm in bms] == [True]*len(bms))):
            print('Ende')
            BMstat = [{'Name': bm.name,'Stopps': bm.stop, 'Länge der insgesamt zurückgelegten Strecke': len(bm.itinerary)} for bm in bms]
            return BMstat, TIME, Schleife
        
        event = hq.heappop(eventq)
        
        Bahnen = [idx for idx, bahn in enumerate(bms) if bahn.name == event.bm]
        if not Bahnen:
            raise Exception('Fehler')
        if len(Bahnen)>1:
            raise Exception('Fehler')
        bm = bms[Bahnen[0]]

        if bm.name != 'dummy':
            itinerary_len = len(bm.itinerary)
            current_node = bm.itinerary[bm.stop % itinerary_len]
            cn_number = [idx for idx, station in enumerate(bm.itinerary) if station == current_node][0] #Current Node Number
        
            next_node = bm.itinerary[(bm.stop + 1) % itinerary_len]
            nn_number = [idx for idx, station in enumerate(bm.itinerary) if station == next_node][0]
            print('Fahrzeug {} ist am Knoten {}, der nächste Halt {}, die aktuelle Zeit: {}'.format(bm.name,ListeNodes[current_node], ListeNodes[next_node], event.time))
            print('und {} Stopp zum {}. Mal, Runden: {}, Rest: {}'.format(bm.name, bm.stop, int(bm.stop/itinerary_len),bm.stop % itinerary_len))
            stop = bm.stop
        else:
            current_node = random.randint(0,len(passengers)-1)
            while current_node == event.p.ziel:
                current_node = random.randint(0,len(passengers)-1)
            print('Fahrzeug {} ist am Knoten {}, der nächste Halt ist zufällig, die aktuelle Zeit: {}'.format(bm.name ,ListeNodes[current_node], event.time))
        
        if event.action == 'load':
            print('Fahrzeug {} load'.format(bm.name))
            print('Fahrzeug {} hat {} Plätze frei'.format(bm.name,bm.capacity - len(bm.n_passengers)))
            dummy = event.p
            umstieg = []
            print('Es warten {} am Knoten {}.'.format(len(passengers[current_node]), ListeNodes[current_node]))
            # Konventionelles Auffüllen
            while len(bm.n_passengers) < bm.capacity:
                if not current_node < len(passengers):
                    raise Exception('Die Länge von passengers {} ist <= current_node {}'.format(len(passengers),current_node))
                waiting = [idx for idx, passenger in enumerate(passengers[current_node]) if passenger.ziel in bm.itinerary]
                if not waiting:
                    break
                passenger = passengers[current_node].pop(waiting[0])
                passenger.bm = bm.name
                
                exit_node = passenger.ziel
                en_number = [idx for idx, station in enumerate(bm.itinerary) if station == exit_node][0]
                
                ListExit = []    
                for i in range((en_number-cn_number)% itinerary_len):
                    ListExit += [ bm.travel_times[(cn_number + i) % itinerary_len] ]
                ExitTime = event.time - 1
                for i in ListExit:
                    ExitTime += (i)
                #print('ExitTime: {}, ListExit: {}'.format(ExitTime,ListExit))
                
                passenger.time = ExitTime
                hq.heappush(bm.n_passengers, ExitTime)
                bm.n_passengers.sort()
                event_neu = Event(ExitTime,'exit', bm.name, passenger)
                if not event_neu.time == passenger.time:
                    raise Exception('Fehler beim Setzen der Event Time')
                if not event_neu.bm == passenger.bm:
                    raise Exception('Fehler beim Setzen des Namens des Beförderungsmittels')
                hq.heappush(eventq, event_neu)
            print('Berechne, wer zukünftig umsteigen soll')
            while len(bm.n_passengers) < bm.capacity:
                if not current_node < len(passengers):
                    raise Exception('Die Länge von passengers {} ist <= current_node {}'.format(len(passengers),current_node))
                waiting = [idx for idx, passenger in enumerate(passengers[current_node]) if passenger.ziel in bm.itinerary]
                umstieg = passengers[current_node][:]
                counter = 0
                while len(waiting) != 0:
                    temp = waiting.pop(0) - counter
                    umstieg.pop(temp)
                    counter += 1
                umstieg = [idx for idx, passenger in enumerate(umstieg) if passenger in passengers[current_node]]
                
                if not passengers[current_node]:
                    break
                if not umstieg:
                    break
                
                p = passengers[current_node][umstieg[0]]
                protect = [bm.name for bm in bms if p.ziel in bm.itinerary and current_node in bm.itinerary]
                
                SortUmstieghilfe = []
                for um in umstieghilfe:
                    if current_node in um:
                        SortUmstieghilfe += [um]
                for um in umstieghilfe:
                    if p.ziel in um:
                        SortUmstieghilfe += [um]
                #print('Protect: ',len(protect) != 0)
                #print('p.ziel {}, current_node {}'.format(p.ziel, current_node))
                #print('SortUmstieghilfe: ', SortUmstieghilfe)
                # Anfang der Schutzfunktion
                if not protect:
                    path = []
                    path0 = []
                    for i in [[i for i in um if i != current_node and i != p.ziel] for um in SortUmstieghilfe]:
                        path0 += i
                    path1 = [current_node]
                    if len(SortUmstieghilfe) ==2:
                        path1 += [i for i in SortUmstieghilfe[1] if i != p.ziel]
                    path2 = []
                    for i in [ [i for i in um] for um in SortUmstieghilfe if current_node in um]:
                        path2 += i
                    #print('path0', path0)
                    #print('path1', path1)
                    #print('path2', path2)
                    
                    PATH0 = [[bm.name for bm in bms if node in bm.itinerary] for node in path0]
                    PATH1 = [[bm.name for bm in bms if node in bm.itinerary] for node in path1]
                    PATH2 = [[bm.name for bm in bms if node in bm.itinerary] for node in path2]
                    #print('PATH0 ',PATH0)
                    #print('PATH1 ',PATH1)
                    #print('PATH2 ',PATH2)
                    #
                    # Hack gilt nur für dieses Szenario
                    #print(cap(PATH0))
                    #print(cap(PATH1))
                    #print(cap(PATH2))
                    
                    #print([len(i) != 0 for i in cap(PATH0)])
                    #print([len(i) != 0 for i in cap(PATH1)])
                    #print([len(i) != 0 for i in cap(PATH2)])
                    
                    if not ([len(i) != 0 for i in cap(PATH0)] == [True]):
                        if not ([len(i) != 0 for i in cap(PATH1)] == [True]):
                            path = path2
                        else:
                            if bm.name in [bm.name for bm in bms[:74]]:
                                path = path1
                            else:
                                break
                    else:
                        path = path0
                    #print('path: {}'.format(path))
                else:
                    break
                # Ende der Schutzfunktion
                
                passenger = passengers[current_node].pop(umstieg.pop(0))
                if not p == passenger: raise Exception('Gedankenfehler')
                
                # Das Setzen des Umstiegs ist vermutlich vom Hack abhängig
                if len(path) == 1:
                    passenger.umstieg = path[0]
                elif len(path) == 2:
                    l = [i for i in path if current_node != i]
                    passenger.umstieg = l[0]
                else:
                    raise Exception('Gedankenfehler')
                
                passenger.bm = bm.name
                exit_node = passenger.umstieg
                #print('exit_node ',exit_node)
                if not [idx for idx, station in enumerate(bm.itinerary) if station == exit_node]:
                    break
                en_number = [idx for idx, station in enumerate(bm.itinerary) if station == exit_node][0]
                
                ListExit = []    
                for i in range((en_number-cn_number)% itinerary_len):
                    ListExit += [ bm.travel_times[(cn_number + i) % itinerary_len] ]
                ExitTime = event.time - 1
                for i in ListExit:
                    ExitTime += (i)
                #print('ExitTime: {}, ListExit: {}'.format(ExitTime,ListExit))
                
                passenger.time = ExitTime
                hq.heappush(bm.n_passengers, ExitTime)
                bm.n_passengers.sort()
                event_neu = Event(ExitTime,'exit', bm.name, passenger)
                if not event_neu.time == passenger.time:
                    raise Exception('Fehler beim Setzen der Event Time')
                if not event_neu.bm == passenger.bm:
                    raise Exception('Fehler beim Setzen des Namens des Beförderungsmittels')
                hq.heappush(eventq, event_neu)
                
            Time = event.time + bm.travel_times[nn_number]
            dummy.time = Time
            hq.heappush(eventq, Event(Time, 'load', bm.name, dummy))
            bm.stop += 1
            print('Es warten noch {} Fahrgäste an Knoten {}'.format(len(passengers[current_node]),ListeNodes[current_node]))
        
        # Generate events for exiting passengers
        elif event.action == 'exit':
            info = [{'Zeit': True}]
            passenger = event.p
            ExitTime = hq.heappop(bm.n_passengers)
            print('PassagierStation:{}, PassagierTime:{}, steigt aus, EventTime {}'.format(passenger.ziel, passenger.time,event.time))
            print('Ist EventTime == PassagierTime: {}'.format(passenger.time == event.time))
            
            if (passenger.time != event.time) and (bm.name != 'dummy'):
                info = [{'Zeit': False}]
                print(passenger)
                bahn = passenger.bm
                Bahnen = [idx for idx, bahn in enumerate(bms) if bahn.name == passenger.bm]
                if not Bahnen:
                    raise Exception('Fehler')
                if len(Bahnen)>1:
                    raise Exception('Fehler')
                bm0 = bms[Bahnen[0]]
                print('Passenger BM:{} CList: {}'.format(bahn, bm0.n_passengers))
                print('Event BM: {} CList: {}'.format(bm.name, bm.n_passengers))
                
            print('Passagier am richtigen Ort: {}'.format([passenger.ziel == current_node, passenger.umstieg == current_node]))
            if not [passenger.ziel == current_node,passenger.umstieg == current_node] == [False]*2:
                    info += [{'Ort': True}]
            if ([passenger.ziel == current_node,passenger.umstieg == current_node] == [False]*2):
                info += [{'Ort': False}]
                if bm.name != 'dummy':
                    print(passenger)
                    bahn = passenger.bm
                    print(bahn,bm.name)
                    print('Passagier mit Ziel: {} wurde falsch berechnet.'.format(passenger.ziel))
                passengers[current_node] += [passenger]
                shuffle0(passengers)
            if not passenger.umstieg == current_node:
                info += [{'Umstieg': False}]
            if passenger.umstieg == current_node:
                info += [{'Umstieg': True}]
                passengers[current_node] += [passenger]
                shuffle0(passengers)
                print('Passagier mit Ziel: {} steigt um.'.format(passenger.ziel))
                passenger.umstieg = 8
            print('Es sind {} Passagiere in der Bahn'.format(len(bm.n_passengers)))
            if bm.name != 'dummy':
                TIME += [{'Info': info ,'EventTime': event.time, 'PassagierTime': passenger.time}]
        else:
            raise Exception('Unknown event action')
        #eventq.sort()
        zykel += 1

In [3]:
graph = nx.Graph()

graph.add_node('Hbf', pos=(10,10))  #0
graph.add_node('INF', pos=(10,30))  #1
graph.add_node('Phil', pos=(30,30))  #2
graph.add_node('Uni', pos=(40,20))  #3
graph.add_node('Bism', pos=(30,10))  #4
graph.add_node('Rohr', pos=(30,0))  #5
graph.add_node('Hand', pos=(15,40))  #6
graph.add_node('Zieg', pos=(50,30))  #7

ListeNodes = list(graph.node)
#Rundfahrt 
graph.add_edge(ListeNodes[0],ListeNodes[1])
graph.add_edge(ListeNodes[1],ListeNodes[0])
graph.add_edge(ListeNodes[1],ListeNodes[2])
graph.add_edge(ListeNodes[2],ListeNodes[1])
##
#graph.add_edge(ListeNodes[0],ListeNodes[2])
##
graph.add_edge(ListeNodes[2],ListeNodes[3])
graph.add_edge(ListeNodes[3],ListeNodes[2])
graph.add_edge(ListeNodes[3],ListeNodes[4])
graph.add_edge(ListeNodes[4],ListeNodes[3])
graph.add_edge(ListeNodes[4],ListeNodes[0])
graph.add_edge(ListeNodes[0],ListeNodes[4])

#Zubringer
graph.add_edge(ListeNodes[4],ListeNodes[5])
graph.add_edge(ListeNodes[5],ListeNodes[4])
graph.add_edge(ListeNodes[1],ListeNodes[6])
graph.add_edge(ListeNodes[6],ListeNodes[1])
graph.add_edge(ListeNodes[3],ListeNodes[7])
graph.add_edge(ListeNodes[7],ListeNodes[3])

ListeNodes = list(graph.node)
ListeColor = [default]*len(ListeNodes)

In [5]:
#""" 74, 40, 40, 40
capa = 12
Color = ['#f0f000','#f000f0','#ff0000','#00ff00']
bms = []
for i in range(1):
    name = "zykel0%02d" % i
    bms += [Vehikel(name, Color[0], 37*capa, [4, 3, 2, 1, 0], [5, 5, 5, 5, 5])]
for i in range(1):
    name = "zykel1%02d" % i
    bms += [Vehikel(name, Color[0], 37*capa, [0, 1, 2, 3, 4], [5, 5, 5, 5, 5])]
for i in range(1):
    name = "zubinger0%02d" % i
    bms += [Vehikel(name, Color[1], 40*capa, [4,5], [5,5])]
for i in range(1):
    name = "zubinger1%02d" % i
    bms += [Vehikel(name, Color[2], 40*capa, [1,6], [5,5])]
for i in range(1):
    name = "zubinger2%02d" % i
    bms += [Vehikel(name, Color[3], 40*capa, [3,7], [5,5])]
#"""

arm0 = 1800 #3600 # 5 # 8
arm1 = 750  #1500 # 1
arm2 = 1000 #2000 # 1
arm3 = 1500 #3000 # 1 # 3
passengers = [[]]*8
for node in range(len(passengers)):
    if node < 5:
        a = []
        for i in range(arm0//5):
            while True:
                ziel = random.randint(0,len(passengers)-1)
                if ziel != node:
                    break
            a += [Passagier(ziel)]
        passengers[node] = a
    elif 5 == node:
        a = []
        for i in range(arm1//2):
            while True:
                ziel = random.randint(0,len(passengers)-1)
                if ziel != node:
                    break
            a += [Passagier(ziel)]
        passengers[node] = a
    elif 6 == node:
        a = []
        for i in range(arm2//2):
            while True:
                ziel = random.randint(0,len(passengers)-1)
                if ziel != node:
                    break
            a += [Passagier(ziel)]
        passengers[node] = a
    elif 7 == node:
        a = []
        for i in range(arm3//2):
            while True:
                ziel = random.randint(0,len(passengers)-1)
                if ziel != node:
                    break
            a += [Passagier(ziel)]
        passengers[node] = a
    else:
        a = []
        passengers[node] = a#"""
shuffle0(passengers)

print(len(passengers))
print([len(i)for i in passengers])
umstieghilfe = [[4,5],[1,6],[3,7]]


# shuffle passengers

sums=0
for node in passengers:
    sums += len(node)
print(sums)

280


In [6]:
#"""
eventq = []
for bm in bms:
    if bm.name != 'dummy':
        event = Event(0, 'load', bm.name ,Passagier(_))
        event.p.bm = event.bm
        event.p.time = event.time
        hq.heappush(eventq, event)#"""


A = solve(eventq, passengers, bms, graph)

In [8]:
KnotenZeit = [{'Knoten': index['PassagierKnoten'],'Zeit': index['Time']} for index in (A[2])]
KnotenZeit_sort0 =[[element for element in KnotenZeit if element['Zeit'] == time][0] for time in range(KnotenZeit[len(KnotenZeit) - 1]['Zeit']+1) if [element for element in KnotenZeit if element['Zeit'] == time]]

fig = plt
fig.figure(figsize=(13.5,7.5))
fig.plot([index['Zeit'] for index in KnotenZeit_sort0], [index['Knoten'] for index in KnotenZeit_sort0])
#fig.plot([index['Zeit'] for index in KnotenZeit_sort0], [sum(index['Knoten']) for index in KnotenZeit_sort0])
fig.title('')
fig.legend(ListeNodes)
fig.xlabel('Zeit')
fig.ylabel('Passagiere')
fig.show()
#fig.savefig('nodes.svg')

In [ ]:
ListeNodes = list(graph.node)
for index in A[2]:
    ListeColor = [default]*len(ListeNodes)
    ListeSize = [sqrt(j)*100 for j in index['PassagierKnoten']]
    if index['Vehiekel'].name != 'dummy':
        ininerary_len = len(index['Vehiekel'].itinerary)
        current_node = index['Vehiekel'].itinerary[index['Vehiekel'].stop % ininerary_len]
        ListeColor[current_node] = index['Vehiekel'].color
    plt.figure(index['Zykel'], figsize=(13.5,7.5))
    calc_pic(graph, ListeColor, ListeSize)
    Text = "Schleifendurchlauf: {}".format(index['Zykel']) + "\n" + "Zeit: {}, Aktion: {}".format(index['Time'],index['Aktion'])
    plt.text(14, 15, Text, fontsize=20)
    plt.show()
    #plt.savefig("img/file%06d.png" % index['Zykel'])

#for i in {$(find . -name 'file*.svg')}; do   echo $i; inkscape -z $i -h 370 -w 550 -d 100 -e $i.png; done
cat *.png | ffmpeg -framerate 5 -f image2pipe -i - animation1.mkv